In [1]:
import itertools
import math
import time
import numpy as np  
import function_wmmse_powercontrol as wf
import random

In [2]:
def create_real(num_H, dis, K,seed):
    np.random.seed(seed)
    CH = np.zeros([num_H, K, K]) + 1j*np.zeros([num_H, K, K])
    pos = np.zeros([num_H,2*K,2])
    distance = np.zeros([num_H,K,K])
    Tx = np.arange(K)
    Rx = np.arange(2*K)[K:]

    for i in range(num_H):
        for j in range(len(Tx)):
            pos[i,Tx[j],:] = np.random.uniform(low=[0.0,0.0], high=[100.0, 100.0], size=(2))
            x = pos[i,Tx[j],0]
            y = pos[i,Tx[j],1]
            dice = np.random.uniform(low= dis[0], high = dis[1])
            rad = np.random.uniform(low=0.0, high= 2* np.pi)
            pos[i,Rx[j],:] = [x+dice*np.cos(rad), y + dice*np.sin(rad)] #assume the number of transmitter and receiver are same

        for j in range(K):
            for k in range(K,2*K):
                    d = np.linalg.norm(pos[i,j,:] - pos[i,k,:])

                    L2 = np.sqrt(1/(1+d**2))
                    distance[i,j,k-K] = d  #record the distance 
                    CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))
                
    return pos, CH, distance

In [3]:
def np_sum_rate(H,p,alpha,var_noise):

    K = H.shape[0]
    p = p.reshape((K,1))                        #align
    H_2 = np.square(H)
    rx_power = np.multiply(H_2, p)
    mask = np.eye(K)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), axis=1)

    interference = np.sum(np.multiply(rx_power, 1 - mask), axis=0) + var_noise

    rate = np.log(1 + np.divide(valid_rx_power, interference))

    w_rate = np.multiply(alpha,rate)
    sum_rate = np.sum(w_rate, axis=1)
    return sum_rate

In [4]:
def allocate_channels(distance, available_channels):
    k= np.shape(distance)[0]
    
    channels = np.zeros(k, dtype=int)
    

    for i in range(k):
        min_dist = np.inf
        closest_neighbor = -1

        # Find the closest neighbor
        for j in range(k):
            #print(j)
            if i != j and distance[j,i] < min_dist:

                min_dist = distance[j,i]
       
                closest_neighbor = j


        # Allocate a channel different from the closest neighbor
        available_channel_list = list(range(available_channels))
        available_channel_list = [x+1 for x in available_channel_list]  #

        if channels[i] ==0:
            channels[i] = np.random.choice(available_channel_list)
        occupiedchannel = channels[i]
        available_channel_list.remove(occupiedchannel)

        if channels[closest_neighbor] == 0:     #flag

            channels[closest_neighbor] = np.random.choice(available_channel_list)
        

    return channels


In [5]:
def optimize_sum_rate(CH,distance):
    channels = allocate_channels(distance[:,:], available_channels)
    channel_1_index = np.where(channels == 1)[0].tolist()
    channel_2_index = np.where(channels == 2)[0].tolist()
    #channel_3_index = np.where(channels == 3)[0].tolist()
    
    CH1_K = len(channel_1_index)
    CH2_K = len(channel_2_index)
    #CH3_K = len(channel_3_index)
    H_CH1 = np.zeros([CH1_K,CH1_K])
    H_CH2 = np.zeros([CH2_K,CH2_K])
    #H_CH3 = np.zeros([CH3_K,CH3_K])

    
    for i in range(CH1_K):
        H_CH1[i,i] = CH[channel_1_index[i],channel_1_index[i]]
        for j in range(i):
            H_CH1[i,j] = CH[channel_1_index[i],channel_1_index[j]]
            H_CH1[j,i] = CH[channel_1_index[j],channel_1_index[i]]

    for i in range(CH2_K):
        H_CH2[i,i] = CH[channel_2_index[i],channel_2_index[i]]
        for j in range(i):
            H_CH2[i,j] = CH[channel_2_index[i],channel_2_index[j]]
            H_CH2[j,i] = CH[channel_2_index[j],channel_2_index[i]]        

    Pmax = 1
    Pini_1 = Pmax*np.ones((1,CH1_K,1))
    Pini_2 = Pmax*np.ones((1,CH2_K,1))
    alpha_1 = np.ones((1,CH1_K))
    alpha_2 = np.ones((1,CH2_K))
    Y_1 = wf.batch_WMMSE2(Pini_1,alpha_1,H_CH1,Pmax,var_noise)
    Y_2 = wf.batch_WMMSE2(Pini_2,alpha_2,H_CH2,Pmax,var_noise)


    sum_rate_1 = np_sum_rate(H_CH1,Y_1,alpha_1,var_noise)
    sum_rate_2 = np_sum_rate(H_CH2,Y_2,alpha_2,var_noise)

    sum_rate = sum_rate_1+sum_rate_2

    return sum_rate
    

In [6]:
num_H = 10
K=10
testseed = 7
var_noise = 0.001
available_channels = 2

In [18]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0

start = time.time()
for i in range(num_H):


    max_sum_rate = optimize_sum_rate(abs(CH[i,:,:]),distance[i,:,:])

    sum_rate += max_sum_rate

sum_rate = sum_rate/num_H
end = time.time()
print('WMMSE time',end-start)

/tmp/ipykernel_268828/2221547859.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


WMMSE time 0.17778253555297852


In [8]:
sum_rate

array([21.515143])

In [9]:
num_H = 100
K=15
testseed = 7
var_noise = 0.001

In [19]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0

start = time.time()
for i in range(num_H):


    max_sum_rate = optimize_sum_rate(abs(CH[i,:,:]),distance[i,:,:])

    sum_rate += max_sum_rate

sum_rate = sum_rate/num_H
end = time.time()
print('WMMSE time',end-start)


/tmp/ipykernel_268828/2221547859.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


WMMSE time 0.17776083946228027


In [11]:
sum_rate

array([28.1554293])

In [12]:
num_H = 100
K=30
testseed = 7
var_noise = 0.001

In [20]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0

start = time.time()
for i in range(num_H):


    max_sum_rate = optimize_sum_rate(abs(CH[i,:,:]),distance[i,:,:])

    sum_rate += max_sum_rate

sum_rate = sum_rate/num_H
end = time.time()
print('WMMSE time',end-start)

/tmp/ipykernel_268828/2221547859.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


WMMSE time 0.1771395206451416


In [14]:
sum_rate

array([40.28032268])

In [15]:
num_H = 100
K=50
testseed = 7
var_noise = 0.001

In [21]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0

start = time.time()
for i in range(num_H):


    max_sum_rate = optimize_sum_rate(abs(CH[i,:,:]),distance[i,:,:])

    sum_rate += max_sum_rate

sum_rate = sum_rate/num_H
end = time.time()
print('WMMSE time',end-start)

/tmp/ipykernel_268828/2221547859.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


WMMSE time 0.17719101905822754


In [17]:
sum_rate

array([52.76447889])

In [14]:
num_H = 100
K=80
testseed = 7
var_noise = 0.001

In [22]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0

start = time.time()
for i in range(num_H):


    max_sum_rate = optimize_sum_rate(abs(CH[i,:,:]),distance[i,:,:])

    sum_rate += max_sum_rate

sum_rate = sum_rate/num_H
end = time.time()
print('WMMSE time',end-start)

/tmp/ipykernel_268828/2221547859.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


WMMSE time 0.17717361450195312


In [20]:
sum_rate

array([65.79215223])